In [1]:
import pyreadr
from langdetect import detect
#from deep_translator import GoogleTranslator
import pandas as pd
from tqdm import tqdm
from transformers import pipeline
import pandas as pd, re


In [2]:
# Path to your file (adjust as needed)
file_path = "Economic data/Final_data/EU_CB_speech.rds"

# Read the RDS file
result = pyreadr.read_r(file_path)

# Extract the first (and usually only) dataframe
df = next(iter(result.values()))

# Display the first few rows
df.head()


,Title,Subtitle,Country,text,Date,All_info
0,European Monetary Union and the Financial Cris...,NaN,DEU,Professor Axel A Weber President of the Deutsc...,2010-10-12,European Monetary Union and the Financial Cris...
1,Financial Literacy and Financial Inclusion: Pr...,Special Address at the 4th OECD/GFLEC Global p...,DEU,1 4th OECD/GFLEC Global policy research sympos...,2017-05-24,Financial Literacy and Financial Inclusion: Pr...
2,The future of the euro area – the perspective ...,"Statement by Claudia Buch, Vice-President Deut...",DEU,Claudia M. Buch Vice-President of Deutsche Bun...,2018-05-21,The future of the euro area – the perspective ...
3,"Competition, Stability, and Efficiency in Fina...",Discussion on a paper by Dean Corbae and Ross ...,DEU,Claudia M. Buch Vice President of the Deutsche...,2018-08-25,"Competition, Stability, and Efficiency in Fina..."
4,Safeguarding the future of monetary union Spee...,Speech delivered at the German Embassy,DEU,Safeguarding the future of monetary union Spee...,2017-10-27,Safeguarding the future of monetary union Spee...


In [11]:
finbert = pipeline("sentiment-analysis", model="ProsusAI/finbert")

def get_financial_sentiment(text):
    if pd.isna(text) or not str(text).strip():
        return None

    text = str(text)
    text = re.sub(r'Page \d+ of \d+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()

    try:
        try:
            lang = detect(text)
        except:
            lang = "en"

        if lang != "en":
            try:
                text = GoogleTranslator(source='auto', target='en').translate(text)
            except Exception as e:
                print(f"Translation failed: {e}")
                return None

        result = finbert(text, truncation=True, max_length=512)
        return result[0]['label']
    except Exception as e:
        print(f"Error processing text: {e}\n{text[:120]}...")
        return None

tqdm.pandas()
df["sentiment_label"] = df["All_info"].progress_apply(get_financial_sentiment)

Device set to use cpu
 37%|███▋      | 2368/6369 [1:58:14<3:19:47,  3.00s/it]


KeyboardInterrupt: 

In [3]:
# Load FinBERT model
finbert = pipeline("sentiment-analysis", model="ProsusAI/finbert")

def get_financial_sentiment(text):
    if pd.isna(text) or not str(text).strip():
        return None

    text = str(text)
    # Clean up extra text patterns and whitespace
    text = re.sub(r'Page \d+ of \d+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()

    try:
        # Run FinBERT sentiment analysis directly (English only)
        result = finbert(text, truncation=True, max_length=512)
        return result[0]['label']
    except Exception as e:
        print(f"Error processing text: {e}\n{text[:120]}...")
        return None

# Apply function with progress bar
tqdm.pandas()
df["sentiment_label"] = df["All_info"].progress_apply(get_financial_sentiment)

Device set to use cpu
100%|██████████| 3452/3452 [2:42:41<00:00,  2.83s/it]  


In [6]:
df.to_csv("CBS_IT_GER_FRA_ESP.csv")

In [7]:
df

,Title,Subtitle,Country,text,Date,All_info,sentiment_label
0,European Monetary Union and the Financial Cris...,NaN,DEU,Professor Axel A Weber President of the Deutsc...,2010-10-12,European Monetary Union and the Financial Cris...,neutral
1,Financial Literacy and Financial Inclusion: Pr...,Special Address at the 4th OECD/GFLEC Global p...,DEU,1 4th OECD/GFLEC Global policy research sympos...,2017-05-24,Financial Literacy and Financial Inclusion: Pr...,neutral
2,The future of the euro area – the perspective ...,"Statement by Claudia Buch, Vice-President Deut...",DEU,Claudia M. Buch Vice-President of Deutsche Bun...,2018-05-21,The future of the euro area – the perspective ...,neutral
3,"Competition, Stability, and Efficiency in Fina...",Discussion on a paper by Dean Corbae and Ross ...,DEU,Claudia M. Buch Vice President of the Deutsche...,2018-08-25,"Competition, Stability, and Efficiency in Fina...",neutral
4,Safeguarding the future of monetary union Spee...,Speech delivered at the German Embassy,DEU,Safeguarding the future of monetary union Spee...,2017-10-27,Safeguarding the future of monetary union Spee...,neutral
...,...,...,...,...,...,...,...
3447,Inflation and monetary policy,"Slides by Philip R. Lane, Member of the Execut...",ECB,,2024-03-22,Inflation and monetary policy<>Slides by Phili...,neutral
3448,R(ising) star?,"Speech by Isabel Schnabel, Member of the Execu...",ECB,SPEECH R(ising) star? Speech by Isabel Sc...,2024-03-20,"R(ising) star?<>Speech by Isabel Schnabel, Mem...",neutral
3449,Debate on: Is the inflation surge over and wha...,"Slides by Philip R. Lane, Member of the Execut...",ECB,,2024-03-20,Debate on: Is the inflation surge over and wha...,neutral
3450,Building confidence in the path ahead,"Speech by Christine Lagarde, President of the ...",ECB,SPEECH Building confidence in the path ahe...,2024-03-20,Building confidence in the path ahead<>Speech ...,positive
